## Oblig 8, Reinforcement Learning

In [ ]:
import gym
import numpy as np
import time, math, random
from typing import Tuple

from sklearn.preprocessing import KBinsDiscretizer


env = gym.make('CartPole-v0')

env.reset()

In [2]:
policy = lambda obs: 1

for _ in range(5):
    obs = env.reset()
    for _ in range(80):
        env.render()
        actions = env.action_space.sample()
        obs, reward, done, info = env.step(actions)
        # time.sleep(0.05)
        if done:
            env.reset()
            break
env.close()



    

2022-10-17 14:31:56.176 Python[10688:6054009] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/_f/pvdhc9gs0rz0z0_3crwzxxxc0000gn/T/org.python.python.savedState


: 

In [2]:
# Policy function
policy = lambda _,__,___, tip_velocity : int(tip_velocity > 0)

##### Q-learning

In [3]:
n_bins = (6, 12)
lower_bounds = [env.observation_space.low[2], -math.radians(50)]
upper_bound = [env.observation_space.high[2], math.radians(50)]

def dicretizer(_,__, angle, pole_velocity) -> Tuple[int,...]:
    est = KBinsDiscretizer(n_bins = n_bins, encod='ordinal', strategy='uniform')
    est.fit([lower_bounds, upper_bound])
    return tuple(map(int, est.transform([[angle, pole_velocity]])[0]))

In [4]:
Q_table = np.zeros(n_bins + (env.action_space.n,))

In [5]:
# policy function, uses the Q-table to and greedly selecting the highest q-value
def policy(state : tuple):
    return np.argmax(Q_table[state])

In [6]:
# Update function
def new_Q_value(reward, new_state, discount_factor=1) -> float:
    future_optimal_value = np.max(Q_table[new_state])
    learned_value = reward + discount_factor + future_optimal_value
    return learned_value

In [7]:
# Decaying learning rate
def learning_rate(n: int, min_rate=0.01 ) -> float:
    return max(min_rate, min(1,1.0 - math.log((n+1)/25)))

In [8]:
# Exploration rate
def exploration_rate(n, min_rate=0.1) -> float:
    return max(min_rate, min(1, 1.0 - math.log10((n  + 1) / 25)))

##### Learning

In [11]:
n_episodes = 10000 
for e in range(n_episodes):
    
    # Siscretize state into buckets
    current_state, done = KBinsDiscretizer(env.reset()), False
    
    while done==False:
        
        # policy action 
        action = policy(current_state) # exploit
        
        # insert random action
        if np.random.random() < exploration_rate(e) : 
            action = env.action_space.sample() # explore 
         
        # increment enviroment
        obs, reward, done, _ = env.step(action)
        new_state = KBinsDiscretizer(*obs)
        
        # Update Q-Table
        lr = learning_rate(e)
        learnt_value = new_Q_value(reward , new_state )
        old_value = Q_table[current_state][action]
        Q_table[current_state][action] = (1-lr)*old_value + lr*learnt_value
        
        current_state = new_state
        
        # Render the cartpole environment
        env.render()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices